In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'a') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [3]:
def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return [line.strip() for line in lines]

In [4]:
import chardet

def read_file_content(file_path):
    # Detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    # Read the file with the detected encoding
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: The file at path {file_path} was not found."
    except UnicodeDecodeError:
        return f"Error: The file at path {file_path} cannot be decoded with the {encoding} encoding."
    except IOError:
        return f"Error: An I/O error occurred while reading the file at path {file_path}."


In [5]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [6]:
TEST_CASE_GENERATOR="""
I want you to act as a software tester.
Your task is to read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "explanation": "Why do you create this test case? How does this test case related to the test scenario inputed?",
  ],
}
If there are more than one test case for this scenario, continue writing other test case in this form.

Rules for generating test steps:
- Describe the test step clearly to make sure each test case is independent, tester do not need to read other information (example: other test case, use case specification) to know how to do that step.
- Avoid references to other test cases or instructions like "do as mentioned."
- If the test case need to be repeated to test with different order, data or case, seperate them to be distinct test cases.
- If the scenario is about testing the displation and there is no flow directly cover that scenario, use only the basic (or main) flow to test it.
- If there are use cases mentioned in extended or included use case, create test case combine use cases, Try to find the connection point of use cases for combination. 
- For test scenarios mentioning navigation in the name, only produce test cases related to the specified navigation method. 
(Example: "Scenario: User navigates to a page by navbar", only produce a test case of user navigates to that page by navbar even though the use case description has many way to navigate to that page)
- For test scenarios not mentioning navigation in the name, do not include any navigation test cases.
- Generate test cases that directly match the scenario name. Choose only one flow to cover the scenario.
- Ensure all actions and objectives match the scenario name.
"""

In [7]:
def sanitize_string(data_str):
    # Replace single quotes with double quotes
    data_str = data_str.replace("'", '"')
    
    # Escape double quotes inside the values
    in_string = False
    sanitized_str = ""
    for i in range(len(data_str)):
        if data_str[i] == '"':
            if in_string and (i == 0 or data_str[i-1] != '\\'):
                sanitized_str += '\\"'
            else:
                sanitized_str += '"'
            in_string = not in_string if data_str[i] == '"' and (i == 0 or data_str[i-1] != '\\') else in_string
        else:
            sanitized_str += data_str[i]
    return sanitized_str

In [8]:
use_case_directory_path = r"D:\Dissertation-GPT\dataset\SpecificationData\Shopping System"
save_path = r"D:\Dissertation-GPT\evaluate\baseline 3 1.8\shopping"

In [9]:
usecase_scenario = {
    "add to cart": [
        "1. Add Single Item Without Options to Cart",
        "2. Add Single Item With Options to Cart - Option Selected",
        "3. Add Multiple Quantities of an Item to Cart",
        "4. Add Item to Cart Without Selecting Available Options",
        "5. Add Item to Cart With Quantity Exceeding Stock",
        "6. Decrease Quantity to Less Than One",
        "7. Add Out of Stock Item to Cart",
        "8. Add Item With Out of Stock Option to Cart",
        "10. Add Multiple Different Items to Cart"
    ],
    "checkout": [
        "1. Successful Checkout with Default Options",
        "2. Checkout with Changed Shipping Address",
        "3. Checkout with New Shipping Address",
        "4. Checkout with Application Voucher Applied",
        "5. Checkout with Shop Voucher Applied",
        "6. Checkout with Changed Payment Method",
        "7. Checkout with Changed Shipping Option",
        "8. Checkout with Note for Shop Added",
        "9. Checkout with No Default Shipping Address",
        "10. Checkout with Incorrect Voucher Code",
        "11. Checkout with Disabled Voucher",
        "12. Checkout with Invalid Shipping Address Input"
    ],
    "product information display": [
        "1. Display Product Information with All Details Present",
        "3. Display Product Information with Out of Stock Status",
        "4. Display Product Information with Special Discount Pricing",
        "5. Expand Customer Reviews and Ratings Section",
        "6. Expand Customer Questions Section",
        "7. Scroll Through Related Product Cards",
        "8. Contact Seller via Seller Contact Option",
        "9. Visit Seller's Store via Store Link",
        "10. Display Product Information with No Customer Reviews",
        "11. Display Product Information with High Volume of Customer Reviews",
        "12. Display Product Information with Negative Customer Reviews",
        "13. Display Product Information with Varied Shipping Options",
        "14. Display Product Information with Free Shipping Option"
    ],
    "purchase": [
        "1. Successful Purchase from Shopping Cart with Multiple Items from Different Stores",
        "2. Successful Purchase from Shopping Cart with Items from a Single Store",
        "3. Successful Direct Purchase from Product Detail Page with Default Options",
        "4. Successful Direct Purchase from Product Detail Page with Selected Options and Adjusted Quantity",
        "5. Attempt to Purchase Out-of-Stock Item from Shopping Cart",
        "6. Attempt to Purchase without Selecting Options in Product Detail Page",
        "7. Attempt to Purchase with Quantity Exceeding Stock in Product Detail Page",
        "8. Attempt to Purchase with Quantity Less Than One in Product Detail Page",
        "9. Attempt to Purchase Out-of-Stock Product Directly from Product Detail Page",
        "10. Attempt to Checkout with Invalid Selected Product in Shopping Cart"
    ],
    "search": [
        "1. Successful Search with Product Name Keyword",
        "2. Successful Search with Partial Product Name Keyword",
        "3. Successful Search with Product Description Keyword",
        "4. Successful Search with Shop Name Keyword",
        "5. Successful Search with Brand Name Keyword",
        "6. Search with Non-Matching Keyword",
        "7. Search Result Pagination Navigation",
        "8. Search and Filter by Most Related",
        "9. Search and Filter by Newest Products",
        "10. Search and Filter by Best Sellers",
        "11. Search and Sort by Price Ascending",
        "12. Search and Sort by Price Descending",
        "13. No Search Results Found Scenario"
    ]
}

In [10]:
for key, value in usecase_scenario.items():
  usecase = key
  usecase_name =usecase
  usecase_path = os.path.join (use_case_directory_path,f"{usecase_name}.txt" )
  for testscenario in value:
    content = "Test scenarios:"+ testscenario+  "\nUse case:" + read_file_content(usecase_path)
    print(content)
    promptExtractCondition = [
      { "role": "system", "content": TEST_CASE_GENERATOR},
      { "role": "user", "content": content}
    ]
    gpt_response = askJSON(promptExtractCondition, client, model)
    print(gpt_response)
    print()
    json_data = json.loads(gpt_response)
    testscenario = testscenario.replace('"','').replace(':','').replace('?','').replace('*','').replace('/','')
    testscenario = testscenario[:150]
    testcase_path = os.path.join (save_path,f"{usecase_name}-{testscenario}.txt" )
    write_json_to_txt_raw(json_data,testcase_path )
  print("f{usecase_name} gen test case complete")

Test scenarios:1. Add Single Item Without Options to Cart
Use case:Use case name: Add item to cart
Description: This feature allows users to add items to their shopping cart for future purchase.
Actor: User
Preconditions: User is logged into their Shopee account.
Postconditions: 
The item is successfully added to the user's shopping cart.
The user can view and manage items in their shopping cart.
Basic Flow:
Step 1: User perform feature 'View a product detail'.
Step 2: User clicks the ""Add to Cart"" button.
Step 3: System confirms that the item has been added to the cart.
Step 5: System adds item to the cart and updates the cart icon to reflect the number of items in the cart.
Alterative flow: 
If product has many options, user choose one available option before adding product to cart.
User can adjust quantity of product before adding product into cart by clicking on minus or plus button next to quantity of the product. 

Exception flow: 
- user cannot add product has many options wit